In [1]:
import torch
from torch import nn
from torch.nn import functional as F
import pandas as pd
from torchviz import make_dot
import matplotlib.pyplot as plt
import torch.optim as optim

from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

In [2]:
path='datasets'
transform = transforms.Compose([
    transforms.ToTensor(),
#     pretraind Imagenet
    transforms.Normalize((0.4915, 0.4823, 0.4468), (0.2470, 0.2435, 0.2616))
])
dataset_train = datasets.CIFAR10(path, train=True, download =True, transform=transform)
dataset_val = datasets.CIFAR10(path, train=False, download =True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
batch_size = 256
train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle= True)
val_loader = DataLoader(dataset_val, batch_size=batch_size, shuffle= True)

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.model = nn.Sequential(nn.Conv2d(in_channels=3, out_channels= 32, kernel_size= 3, padding=1),
                                   nn.ReLU(),
                                   nn.BatchNorm2d(32),
                                   nn.Conv2d(32,64,3, padding=1),
                                   nn.MaxPool2d(stride=1, kernel_size=2),
                                   nn.ReLU(),
                                   nn.BatchNorm2d(64),
                                   nn.Flatten(),
                                   nn.LazyLinear(128),
                                   nn.BatchNorm1d(128),
                                   nn.LazyLinear(10)
                                  )
    def forward(self, x):
        x= self.model(x)
        return x

In [5]:
model=Net()
model

/home/huynv/.local/lib/python3.9/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Net(
  (model): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): MaxPool2d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
    (5): ReLU()
    (6): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Flatten(start_dim=1, end_dim=-1)
    (8): LazyLinear(in_features=0, out_features=128, bias=True)
    (9): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LazyLinear(in_features=0, out_features=10, bias=True)
  )
)

In [6]:
def training(n_epochs, model, loss_fn, optimizer, train_loader, val_loader ):
    losses = []
    for epoch in range(n_epochs):
        loss_train =0
        for x, y in tqdm(train_loader):
            y_hat = model(x)
            loss = loss_fn(y_hat, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_train += loss.item()
        losses.append(loss_train)
        c=0
        with torch.no_grad():
            for x, y_true in val_loader:
                pred = model(x)
                _, y_pred = torch.max(pred, 1)
                c += (y_pred==y_true).squeeze().sum()
        print(f'Epoch: {epoch +1} Train_loss {loss_train/len(train_loader)}, Val_acc {c/len(dataset_val)} ')

In [8]:
model = Net()
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

training(
    n_epochs = 30,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
    val_loader = val_loader
)

/home/huynv/.local/lib/python3.9/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 196/196 [02:07<00:00,  1.54it/s]


Epoch: 1 Train_loss 1.2663656996220958, Val_acc 0.6474999785423279 


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 196/196 [02:06<00:00,  1.54it/s]


Epoch: 2 Train_loss 0.8922649631086661, Val_acc 0.6827999949455261 


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 196/196 [02:06<00:00,  1.55it/s]


...

Epoch: 28 Train_loss 0.007326356916954475, Val_acc 0.6726999878883362 


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 196/196 [02:02<00:00,  1.60it/s]


Epoch: 29 Train_loss 0.04115118886396403, Val_acc 0.6517999768257141 


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 196/196 [02:02<00:00,  1.61it/s]


Epoch: 30 Train_loss 0.1318307662648814, Val_acc 0.6603000164031982 
